In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

import time
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2 
import itertools

import csv
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
#from vgg16_2 import VGG_16

from vgg16_2 import VGG_16_2


import argparse
from data_loader_half_1  import get_loader
from data_loader_half_1  import get_dataset

import math

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import time
import shutil
import os
import torch.nn.functional as F

# Loading dataset

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
celeba_image_dir = '/home/name/celeba/stargan/data/celeba/images'
attr_path = '/home/name/celeba/stargan/data/celeba/list_attr_celeba.txt'
id_path = '/home/name/celeba/stargan/data/celeba/identity_CelebA.txt'
selected_attrs = ['Black_Hair']
celeba_crop_size = 224#178
image_size = 224 #128
batch_size = 30#100
num_workers = 0
mode1 = 'train' #test or train
mode2 = 'test'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #"cuda:0" if torch.cuda.is_available() else 
torch.set_default_tensor_type('torch.cuda.FloatTensor')


 



In [3]:
celeba_loader_train, data_train = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode1, num_workers)
celeba_loader_test, data_test = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...
Finished preprocessing the CelebA dataset...


In [4]:
celeba_loader_test2, data_test2 = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, 1,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...


In [5]:
print(len(data_train))
print(len(data_test))

88374
12625


# Validate

In [6]:
def validate(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()
        
        del im
        del preds
        
        del values
        
        print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)


In [20]:
def validate_per_example(model, mask):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        preds, _ = model(im, mask)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



In [8]:
def validate_per_example_mask(model, mask):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        im = im * mask
        preds = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



# Noisy Network

In [12]:

class NoisyActivation(nn.Module):
    def __init__(self,  given_locs, given_scales, min_scale, max_scale):
        super(NoisyActivation, self).__init__()
        size = given_scales.shape
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.given_locs = given_locs 
        self.given_scales = given_scales
        self.locs = nn.Parameter(torch.Tensor(size).copy_(self.given_locs))         
        self.rhos = nn.Parameter(torch.ones(size)-5) #-inf


        self.normal = torch.distributions.normal.Normal(0,1)
        self.rhos.requires_grad = False
        self.locs.requires_grad = False
        
    def scales(self):
        return (1.0 +torch.tanh(self.rhos))/2*(self.max_scale-self.min_scale) +self.min_scale             
    
    def sample_noise(self, mask):
        epsilon = self.normal.sample(self.rhos.shape)
        return self.locs + self.scales() * epsilon*mask
                                 
                               
                            
    def forward(self, input, mask):
        noise = self.sample_noise(mask)
        return (input)*mask + noise



class vgg_syn(nn.Module):

    def __init__(self, model, min_scale,max_scale, given_locs, given_scale):
        super(vgg_syn, self).__init__()
        

        self.intermed = NoisyActivation( given_locs, given_scale, min_scale, max_scale)
        #self.model_pt2 =  torch.nn.Sequential(*(list(model_features)))
        self.model_pt3 = model
        #self.components = components
        
        for param in self.model_pt3.parameters():
            param.requires_grad = False

                
        self.intermed.rhos.requires_grad = False
        self.intermed.locs.requires_grad = False
                                 
    def forward(self, img,mask):
                                 
        
        x = self.intermed(img, mask)
        noisy = x.detach()
         

        x = self.model_pt3(x)                                 

        return x, noisy
    
                                 
 






In [14]:

model = models.resnet18(num_classes=2)

model.load_state_dict(torch.load("resnet-hair-90l0",  map_location=device))


criterion = nn.NLLLoss()
mus = torch.zeros((3,224,224))
scale = torch.ones((3,224,224))*0.001
model_syn = vgg_syn(model ,0.0,5 ,mus, scale )
our = torch.load('celeba-vgg-hair-avg-cloak',  map_location=device)

In [15]:
model_syn.intermed.rhos.copy_(our['intermed.rhos'])
#model_syn.intermed.locs.copy_(our['intermed.locs'])

Parameter containing:
tensor([[[3.3576, 3.4472, 3.3453,  ..., 3.4158, 3.4429, 3.3928],
         [3.3400, 3.4145, 3.3235,  ..., 3.3312, 3.4148, 3.3677],
         [3.4122, 3.4117, 3.3880,  ..., 3.3151, 3.3966, 3.3646],
         ...,
         [3.4327, 3.4533, 3.4562,  ..., 3.5758, 3.5906, 3.4998],
         [3.4303, 3.4600, 3.4133,  ..., 3.6174, 3.5876, 3.4846],
         [3.3558, 3.3965, 3.3908,  ..., 3.5534, 3.5449, 3.4801]],

        [[3.3622, 3.3467, 3.3938,  ..., 3.3017, 3.4115, 3.3935],
         [3.3414, 3.4309, 3.4864,  ..., 3.3643, 3.4530, 3.4510],
         [3.4346, 3.3907, 3.3934,  ..., 3.3055, 3.3866, 3.4437],
         ...,
         [3.3944, 3.3648, 3.4524,  ..., 3.6065, 3.5487, 3.5359],
         [3.3628, 3.4473, 3.4059,  ..., 3.5691, 3.5629, 3.5140],
         [3.3724, 3.4298, 3.3402,  ..., 3.5112, 3.5114, 3.4430]],

        [[3.2711, 3.2930, 3.3877,  ..., 3.2963, 3.3783, 3.3519],
         [3.4069, 3.3724, 3.4817,  ..., 3.3054, 3.3402, 3.4033],
         [3.3700, 3.4120, 3.4251,  .

In [17]:
#mult_mask =  torch.ones(model_syn.intermed.rhos.shape)
mult_mask = torch.where(model_syn.intermed.scales()>4.5, torch.zeros(model_syn.intermed.scales().shape), torch.ones(model_syn.intermed.scales().shape))

In [208]:
criterion = nn.CrossEntropyLoss().to(device)

In [21]:
validate_per_example(model_syn , mult_mask) #91.60

1.0
1.0
1.0
1.0
0.8
0.8333333333333334
0.8571428571428571
0.75
0.7777777777777778
0.8
0.8181818181818182
0.8333333333333334
0.8461538461538461
0.8571428571428571
0.8666666666666667
0.8125
0.8235294117647058
0.7777777777777778
0.7894736842105263
0.8
0.7619047619047619
0.7727272727272727
0.782608695652174
0.7916666666666666
0.8
0.8076923076923077
0.8148148148148148
0.8214285714285714
0.8275862068965517
0.8333333333333334
0.8387096774193549
0.84375
0.8484848484848485
0.8235294117647058
0.8285714285714286
0.8333333333333334
0.8108108108108109
0.8157894736842105
0.8205128205128205
0.8
0.7804878048780488
0.7619047619047619
0.7674418604651163
0.7727272727272727
0.7777777777777778
0.782608695652174
0.7872340425531915
0.7916666666666666
0.7959183673469388
0.8
0.803921568627451
0.8076923076923077
0.8113207547169812
0.8148148148148148
0.8181818181818182
0.8214285714285714
0.8245614035087719
0.8275862068965517
0.8305084745762712
0.8333333333333334
0.8360655737704918
0.8387096774193549
0.8412698412

0.7635574837310195
0.7640692640692641
0.7645788336933045
0.7650862068965517
0.7655913978494624
0.7660944206008584
0.7665952890792291
0.7670940170940171
0.767590618336887
0.7680851063829788
0.7685774946921444
0.7690677966101694
0.7695560253699789
0.7679324894514767
0.7684210526315789
0.7689075630252101
0.7693920335429769
0.7698744769874477
0.7682672233820459
0.76875
0.7692307692307693
0.7697095435684648
0.7701863354037267
0.7706611570247934
0.7690721649484537
0.7674897119341564
0.7659137577002053
0.764344262295082
0.7627811860940695
0.763265306122449
0.7637474541751528
0.7642276422764228
0.7626774847870182
0.7631578947368421
0.7636363636363637
0.7641129032258065
0.7645875251509054
0.7650602409638554
0.7655310621242485
0.766
0.7664670658682635
0.7669322709163346
0.7654075546719682
0.7658730158730159
0.7663366336633664
0.766798418972332
0.7672583826429981
0.765748031496063
0.7662082514734774
0.7666666666666667
0.7651663405088063
0.763671875
0.7621832358674464
0.7626459143968871
0.76310679

0.7605633802816901
0.7608225108225108
0.7610810810810811
0.7602591792656588
0.7605177993527508
0.7607758620689655
0.7610333692142088
0.7612903225806451
0.7604726100966702
0.759656652360515
0.759914255091104
0.7601713062098501
0.760427807486631
0.7606837606837606
0.7609391675560299
0.7601279317697228
0.7603833865814696
0.7595744680851064
0.7598299681190224
0.7600849256900213
0.7592788971367974
0.7595338983050848
0.7597883597883598
0.7600422832980972
0.7602956705385427
0.760548523206751
0.7608008429926238
0.7610526315789473
0.7613038906414301
0.7615546218487395
0.7607555089192025
0.7610062893081762
0.7602094240837697
0.7604602510460251
0.7607105538140021
0.7609603340292276
0.7612095933263816
0.7614583333333333
0.7617065556711758
0.761954261954262
0.7622014537902388
0.7624481327800829
0.7626943005181347
0.7619047619047619
0.7611168562564633
0.7613636363636364
0.7616099071207431
0.7608247422680412
0.761071060762101
0.7613168724279835
0.7615621788283659
0.7618069815195072
0.7610256410256411

0.7617328519855595
0.7619047619047619
0.7613554434030281
0.7615273775216138
0.761699064074874
0.7618705035971223
0.7613227893601725
0.7614942528735632
0.7609475951184493
0.7611190817790531
0.7612903225806451
0.7607449856733525
0.7609162491052255
0.7610872675250357
0.7612580414581844
0.7607142857142857
0.7608850820842256
0.761055634807418
0.7612259444048468
0.7613960113960114
0.7615658362989324
0.7617354196301565
0.7611940298507462
0.7613636363636364
0.7608232789212207
0.7609929078014185
0.7611622962437987
0.7613314447592068
0.7615003538570417
0.7609618104667609
0.76113074204947
0.7612994350282486
0.7614678899082569
0.7609308885754584
0.7610993657505285
0.7605633802816901
0.7600281491907108
0.760196905766526
0.7603654251581167
0.7605337078651685
0.76
0.7594670406732118
0.759635599159075
0.7598039215686274
0.7599720083974808
0.7594405594405594
0.7589098532494759
0.7583798882681564
0.7585484996510816
0.7587168758716876
0.7588850174216027
0.7590529247910863
0.7592205984690327
0.75938803894

0.7563616675690309
0.7564935064935064
0.756625202812331
0.7567567567567568
0.7568881685575365
0.7570194384449244
0.7571505666486779
0.7572815533980582
0.7574123989218329
0.7575431034482759
0.7571351642434033
0.7572658772874058
0.757396449704142
0.7569892473118279
0.7571198280494358
0.757250268528464
0.7568438003220612
0.7569742489270386
0.7565683646112601
0.7566988210075026
0.7568291376539904
0.7569593147751607
0.7565543071161048
0.7566844919786097
0.7568145376803849
0.7569444444444444
0.7570742124933262
0.7572038420490929
0.7573333333333333
0.7574626865671642
0.7575919019712307
0.7577209797657082
0.7578499201703034
0.7579787234042553
0.7581073896863371
0.7582359192348566
0.7578332448220924
0.7579617834394905
0.7575596816976128
0.7576882290562036
0.7578166401695814
0.7579449152542372
0.7580730545262043
0.7576719576719577
0.7578001057641459
0.7579281183932347
0.7580559957739038
0.7581837381203802
0.7577836411609499
0.7579113924050633
0.7580390089615182
0.7581664910432033
0.7582938388625

...

0.7653585926928281
0.765422077922078
0.7654855288071409
0.7655489453758788
0.7653419843200865
0.7654054054054054
0.7654687922183194
0.7655321447866018
0.7655954631379962
0.7653887688984882
0.7654520917678812
0.7655153804641123
0.7655786350148368
0.7656418554476807
0.7657050417902399
0.7654986522911051
0.7652923740231743
0.7653556034482759
0.7651494748182063
0.7652127086698977
0.765006729475101
0.765069967707212
0.7651331719128329
0.765196342119419
0.7649905888679752
0.7650537634408602
0.7651169040580489
0.7651800107469102
0.7652430835347838
0.7653061224489796
0.7653691275167785
0.7654320987654321
0.7654950362221626
0.7655579399141631
0.7656208098685975
0.7656836461126005
0.7657464486732779
0.7658092175777064
0.7656040717921243
0.7653990358864489
0.765194109772423
0.7652569593147751
0.7653197752207653
0.7653825575173889
0.7654453062316128
0.76524064171123
0.7653033948142208
0.7653661143773384
0.7654288004274646
0.7654914529914529
0.7652870493991989
0.7653497063534437
0.7654123298638911


0.7628097185470291
0.7628667628667629
0.7629237797547488
0.7629807692307692
0.7630377313145879
0.7630946660259491
0.7631515733845784
0.7632084534101825
0.763265306122449
0.7630820931349016
0.7631389488840893
0.7631957773512476
0.7632525785560086
0.7630695443645084
0.7628865979381443
0.7627037392138063
0.7625209681284447
0.7623382846190704
0.7623952095808383
0.7624521072796935
0.7622695714627723
0.7623264719961704
0.7621440536013401
0.7619617224880383
0.7620186558239656
0.7620755619320899
0.7621324408319388
0.762189292543021
0.7622461170848268
0.7623029144768275
0.7623596847384763
0.7624164278892073
0.7624731439484363
0.7625298329355609
0.7625864948699594
0.7626431297709924
0.7624612449320296
0.762279446828803
0.7623361144219308
0.7621544327931363
0.7622111031689301
0.7622677465459743
0.762324362943558
0.7621428571428571
0.761961437752916
0.7620180866254165
0.7618367832500594
0.7616555661274976
0.7617122473246135
0.7617689015691869
0.7618255288804374
0.7618821292775665
0.761938702779757

0.7633686945226239
0.7634199134199134
0.7634711101493183
0.7635222847252272
0.763357127406446
0.7631920415224913
0.7632432432432432
0.7632944228274967
0.7633455802896045
0.7633967156439067
0.7634478289047311
0.7634989200863931
0.7635499892031958
0.7636010362694301
0.763652061299374
0.763703064307294
0.7637540453074434
0.7638050043140638
0.7636402846668104
0.7636912462268219
0.7637421858159086
0.7637931034482759
0.7638439991381167
0.7638948728996122
0.7639457247469309
0.7639965546942291
0.7640473627556512
0.7640981489453293
0.7641489132773832
0.7641996557659209
0.7642503764250377
0.7643010752688172
0.7643517523113309
0.764402407566638
0.7644530410487858
0.7645036527718092
0.7643394199785177
0.7643900343642611
0.7644406270130986
0.7644911979390296
0.7645417471560421
0.7645922746781116
0.7646427805192019
0.7646932646932647
0.7647437272142398
0.7647941680960549
0.7646302250803858
0.7646806686669524
0.7647310906363831
0.7645672664952871
0.7644035125294496
0.7644539614561028
0.76429030186255

0.7608738437315489
0.7609208972845336
0.7607711981113515
0.7608182533438238
0.7608652900688299
0.7609123082972866
0.7609593080401023
0.7608097484276729
0.7608567498526233
0.7609037328094302
0.7609506973089766
0.7609976433621367
0.7608482230512468
0.7608951707891637
0.7609421000981355
0.760989010989011
0.761035903472631
0.7610827775598273
0.7611296332614238
0.7611764705882353
0.76102724955891
0.7608780870246962
0.7609249461101313
0.7609717868338558
0.7610186092066601
0.7610654132393263
0.7611121989426277
0.7611589663273297
0.7612057154041887
0.761252446183953
0.761103502250049
0.7609546165884195
0.7610013690592607
0.7608525615956199
0.7607038123167156
0.7605551211884285
0.7606019151846786
0.7606486908948809
0.7606954483297519
0.7607421875
0.760788908416325
0.7608356110894182
0.760687097403865
0.7605386416861827
0.7605853658536585
0.7606320717908701
0.7606787595084845
0.7607254290171607
0.7607720803275493
0.7606237816764132
0.7606704346131359
0.7607170693686672
0.7607636859536333
0.76061

KeyboardInterrupt: 